# Design template: Model specification

This notebook hosts a template for model specification in OpenSourceEconomics. As an example, we consider the model of a simple $AR(1)$ process. 

\begin{align*}
x_{t + 1} = \rho x_t + \epsilon_t,\quad\text{for}\quad t = 1, ..., T
\end{align*}

with $x_t = 0$ and $\epsilon_t \sim N(0, 1)$. There are two remaiing parameters, $\rho$ the level of serial correlation and $T$ the number of time periods. We can think of $\rho$ as parameter to be estimated and thus updated during an estimation and $T$ as set throughout.

In [1]:
from collections import namedtuple
import pathlib
import copy

import numpy as np
import yaml

# We use a namedtuple as a base class and define all valid fields
# here.
fields = ['rho', 'periods']
ModelBase = namedtuple('ModelBase', ' '.join(fields))

class ModelSpec(ModelBase):
    """Model Specifiction.
     
    Class that contains all the information required to simulate
    a specified AR(1) process. It is the one an central place
    that contains this information throughout. It is an extended
    version of a namedtuple and thus ensures that model specification
    remains immutable.
     
    Attributes:
        rho: a float indicating the degree of serial correlation
        periods: an integer for the length of the time horizon
     
    """
    def copy(self):
        return copy.deepcopy(self)
    
    def as_dict(self):
        return self._asdict()
    
    def replace(self, *args, **kwargs):
        return self._replace(*args, **kwargs)
    
    def to_yaml(self, fname='test.yml'):
        with open(fname, 'w') as out_file:
            yaml.dump(self._asdict(), out_file)
            
    def validate(self):
        """This method validates the model specification. All validation is done here and no further checks 
        are necessary later in the program for the immutable parameters describing the model.
        
        The design ensures that all fields require exlicit checks.
        """
        for field in self._fields:
            if field == 'periods':
                attr = getattr(self, field)
                assert isinstance(attr, int)
                assert attr > 0
            elif field == 'rho':
                attr = getattr(self, field)
                assert isinstance(attr, float)
            else:
                raise NotImplementedError('validation of {:} missing'.format(field))
    
    def __repr__(self):
        """Provides a string representation of the model speficiation for
        quick visual inspection.
        """
        str_ = ''
        for field in self._fields:
            str_ +='{:}: {:}\n'.format(field, getattr(self, field))
        return str_
    
    def __eq__(self, other):
        """Check the equality of two model specifications.
        
        Returns true if two model specificaions have the same fields defined
        and all have the same value.
        
        Args:
            other: A ModelSpec instance.
        
        Returns:
            A boolean corresponding to equality of specifications.
        
        """
        assert isinstance(other, type(self))
        assert set(spec_1._fields) == set(spec_2._fields)
        for field in self._fields:
            if getattr(self, field) != getattr(other, field):
                return False
        return True
 
    def __ne__(self, other):
        """Check the inequality of two model specification."""
        return not self.__eq__(other)
    
def generate_random_model_specification(constr=None):
    """Create a random model specifaction.
    
    Creates a random specification of the model which is useful 
    for testing the robustness of implementation and testing
    in general.
    
    Args:
        constr: A dictionary that contains the requested constrains.
            The keys correspond to the field that is set to the value
            field.
            
            {'periods': 4, 'rho': 0.4}
    """
    def process_constraints(constr):
        """Impose a constraint on initialization dictionary.
        
        This function processes all constraints passed in by the user 
        for the random model specification.
        
        Args:
            constr: A dictionary which contsins 
        
        """ 
        if constr is None:
            constr = dict()
            
        if constr.get('periods'):
            init_dict['periods'] = constr['periods']
        if constr.get('rho'):
            init_dict['rho'] = constr['rho']
    
    init_dict = dict()
    init_dict['rho'] = np.random.uniform(0.01, 0.99)
    init_dict['periods'] = np.random.randint(1, 10)
        
    process_constraints(constr)
        
    return init_dict
    
def get_model_obj(source=None, constr=None):
    """Get model specification.
    
    This is a factory method to create a model spefication from
    a variety of differnt input types.
    
    Args:
        input: str, dictionary, None specifying the input for
            for the model specification.
        constr: A dictionary with the constraints imposed
            on a random initialization file.
    
    Returns:
        An instance of the ModelSpec class with the model
        specifiation.
    """    
    # We want to enforce the use of Path objects going forward.
    if isinstance(source, str):
        source = pathlib.Path(source)
 
    if isinstance(source, dict):
        model_spec = ModelSpec(**source)
    elif isinstance(source, pathlib.PosixPath):
        model_spec = ModelSpec(**yaml.load(open(source, 'r'), Loader=yaml.FullLoader))
    elif source is None:
        model_spec = ModelSpec(**generate_random_model_specification(constr))
    else:
        raise NotImplementedError
    
    # We validate our model specification once and for all.
    model_spec.validate()

    return model_spec

## Use cases 

We want to explore some use cases with a basic model and translate them into tests.

We can specify a model programmatically using a dictionary.

In [2]:
init_dict = {'rho': 0.5, 'periods': 10}
get_model_obj(init_dict);

As an alternative we can also read it in from a *.yml* file.

In [3]:
# %load model_spec.yml
periods: 2
rho: 0.5
    
get_model_obj('model_spec.yml');

* We want to be able to update the parameters of the model specification during an optimization.

In [4]:
spec_1 = get_model_obj(None)
spec_2 = spec_1.replace(rho=0.9)

* We want to easily compare different model specifications.

In [5]:
spec_1 = get_model_obj(None)
assert spec_1 != spec_1.replace(rho=0.9)

spec_1 = get_model_obj(None)
spec_2 = spec_1.copy()
assert spec_1 == spec_2

* We want to be able to go back and forth between the different ways a model is stored.

In [6]:
for _ in range(100):
    spec_1 = get_model_obj(None)
    spec_1.to_yaml()

    spec_2 = get_model_obj('test.yml')
    assert spec_1 == spec_2

    spec_3 = get_model_obj(pathlib.Path('test.yml'))
    assert spec_1 == spec_3
    
    spec_4 = get_model_obj(spec_1.as_dict())
    assert spec_1 == spec_4

* We want to easily validate the integrity of our model specification.

In [7]:
for _ in range(100):
    spec = get_model_obj(None)
    spec.validate()

* We want to easily access all fields.

In [8]:
for _ in range(100):
    spec = get_model_obj(None)
    for field in spec._fields:
        field, getattr(spec, field)

* We want to easily inspect the model specification.

In [9]:
spec = get_model_obj(None)
print(spec)

rho: 0.3973099843123803
periods: 7



* We do not want to change parts of our model specification by accident.

In [10]:
spec = get_model_obj(None)

# We cannot change a field already defined.
with np.testing.assert_raises(AttributeError):
    spec.periods = 1

# We cannot add a field dynamically.
spec_1 = spec.copy()
spec.period = 1 
# Note that the statement above does not throw an error though.
# It simply does not have any effect
assert set(spec._fields) == set(['rho', 'periods'])

## Integration

This model class can then be used to work with the specified model.

In [12]:
def simulate(model_spec):
    """This function simulates a simple AR(1) process."""
    assert isinstance(model_spec, ModelSpec)
    
    sequence = np.tile(np.nan, model_spec.periods)
    sequence[0] = 0
    for i in range(1, model_spec.periods):
        sequence[i] = np.random.normal() + model_spec.rho * sequence[i - 1]
    return sequence

model_spec = get_model_obj('test.yml')
simulate(model_spec);

We can then combine the testing features.

In [13]:
for _ in range(100):
    rslt = simulate(get_model_obj(None))
    assert np.isnan(rslt).any() == False

# Additional resources

* https://paramtools.readthedocs.io is an attempt to conduct a similar effort. It appears general in the sense that the constraints we define in the validate() method are specified in an external file thus increaseing portability across different model types. This is worthwile to follow along.